# Ollama + OpenAI + Python

## 1. Укажете името на модела

Ако сте избрали различен модел от "phi3:mini", променете стойността в клетката по-долу. Тази променлива ще се използва в кода из целия тетраден файл.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Настройка на Open AI клиента

Обикновено клиентът на OpenAI се използва с OpenAI.com или Azure OpenAI за взаимодействие с големи езикови модели.  
Въпреки това, той може да се използва и с Ollama, тъй като Ollama предоставя съвместим с OpenAI крайна точка на "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Генериране на завършване на чат

Сега можем да използваме OpenAI SDK, за да генерираме отговор за разговор. Тази заявка трябва да създаде хайку за котки:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Създаване на подсказки

Първото съобщение, изпратено до езиковия модел, се нарича "системно съобщение" или "системна подсказка" и задава общите инструкции за модела.  
Можете да предоставите своя собствена персонализирана системна подсказка, за да насочите езиковия модел да генерира изход по различен начин.  
Модифицирайте `SYSTEM_MESSAGE` по-долу, за да отговаря като вашия любим известен герой от филм или телевизионен сериал, или почерпете вдъхновение за други системни подсказки от [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

След като персонализирате системното съобщение, задайте първия въпрос на потребителя в `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Примери с малко на брой примери

Друг начин да насочите езиков модел е да предоставите "малко на брой примери", поредица от примерни въпроси и отговори, които показват как трябва да реагира.

Примерът по-долу се опитва да накара езиков модел да действа като помощник-преподавател, като предоставя няколко примера за въпроси и отговори, които помощник-преподавателят би могъл да даде, и след това задава на модела въпрос, който студент би могъл да зададе.

Опитайте първо, а след това модифицирайте `SYSTEM_MESSAGE`, `EXAMPLES` и `USER_MESSAGE` за нов сценарий.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Извличане с допълнително генериране

RAG (Извличане с допълнително генериране) е техника, която позволява на езиков модел да отговаря точно на въпроси за конкретна област, като първо извлича релевантна информация от източник на знания и след това генерира отговор въз основа на тази информация.

Предоставили сме локален CSV файл с данни за хибридни автомобили. Кодът по-долу чете CSV файла, търси съвпадения с въпроса на потребителя и след това генерира отговор въз основа на намерената информация. Имайте предвид, че това ще отнеме повече време от предишните примери, тъй като изпраща повече данни към модела. Ако забележите, че отговорът все още не е основан на данните, можете да опитате системно инженерство или да използвате други модели. По принцип, RAG е по-ефективен с по-големи модели или с фино настроени версии на SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
